Pablo Alejandro Mendez Morales #19195

# Análisis de Malware

In [1]:
import pefile as pe
from rich import print
from rich.tree import Tree
from rich.table import Table
from hashlib import file_digest

## Parte 1 - Análisis estático

**1. Utilice la herramienta pefile para examinar el PE header y obtenga las DLL y las APIs que cada uno de los ejecutables utilizan. ¿Qué diferencias observa entre los ejemplos? ¿Existe algún indicio sospechoso en la cantidad de DLLs y las APIs llamadas?**

In [2]:
sample_qwrty_dk2 = pe.PE("./MALWR/sample_qwrty_dk2")
sample_vg655_25th = pe.PE("./MALWR/sample_vg655_25th.exe")

In [3]:
def inspect_pe(name: str, executable):
    tree = Tree("[bold red]" + name)
    for entry in executable.DIRECTORY_ENTRY_IMPORT:
        entry_node = tree.add("[bold green]" + entry.dll.decode('ascii'))
    
        for function in entry.imports:
            entry_node.add("[purple]" + function.name.decode('ascii'))
    
    print(tree)

In [4]:
 inspect_pe("sample_qwrty_dk2", sample_qwrty_dk2)

sample_qwrty_dk2
├── KERNEL32.DLL
│   ├── LoadLibraryA
│   ├── ExitProcess
│   ├── GetProcAddress
│   └── VirtualProtect
├── MSVCRT.dll
│   └── atol
├── SHELL32.dll
│   └── SHChangeNotify
├── USER32.dll
│   └── LoadStringA
└── WS2_32.dll
    └── closesocket

In [5]:
 inspect_pe("sample_vg655_25th.exe", sample_vg655_25th)

sample_vg655_25th.exe
├── KERNEL32.dll
│   ├── GetFileAttributesW
│   ├── GetFileSizeEx
│   ├── CreateFileA
│   ├── InitializeCriticalSection
│   ├── DeleteCriticalSection
│   ├── ReadFile
│   ├── GetFileSize
│   ├── WriteFile
│   ├── LeaveCriticalSection
│   ├── EnterCriticalSection
│   ├── SetFileAttributesW
│   ├── SetCurrentDirectoryW
│   ├── CreateDirectoryW
│   ├── GetTempPathW
│   ├── GetWindowsDirectoryW
│   ├── GetFileAttributesA
│   ├── SizeofResource
│   ├── LockResource
│   ├── LoadResource
│   ├── MultiByteToWideChar
│   ├── Sleep
│   ├── OpenMutexA
│   ├── GetFullPathNameA
│   ├── CopyFileA
│   ├── GetModuleFileNameA
│   ├── VirtualAlloc
│   ├── VirtualFree
│   ├── FreeLibrary
│   ├── HeapAlloc
│   ├── GetProcessHeap
│   ├── GetModuleHandleA
│   ├── SetLastError
│   ├── VirtualProtect
│   ├── IsBadReadPtr
│   ├── HeapFree
│   ├── SystemTimeToFileTime
│   ├── LocalFileTimeToFileTime
│   ├── CreateDirectoryA
│   ├── GetStartupInfoA
│   ├── SetFilePointer
│   ├── SetFileTime
│   ├── GetComputerNameW
│   ├── GetCurrentDirectoryA
│   ├── SetCurrentDirectoryA
│   ├── GlobalAlloc
│   ├── LoadLibraryA
│   ├── GetProcAddress
│   ├── GlobalFree
│   ├── CreateProcessA
│   ├── CloseHandle
│   ├── WaitForSingleObject
│   ├── TerminateProcess
│   ├── GetExitCodeProcess
│   └── FindResourceA
├── USER32.dll
│   └── wsprintfA
├── ADVAPI32.dll
│   ├── CreateServiceA
│   ├── OpenServiceA
│   ├── StartServiceA
│   ├── CloseServiceHandle
│   ├── CryptReleaseContext
│   ├── RegCreateKeyW
│   ├── RegSetValueExA
│   ├── RegQueryValueExA
│   ├── RegCloseKey
│   └── OpenSCManagerA
└── MSVCRT.dll
    ├── realloc
    ├── fclose
    ├── fwrite
    ├── fread
    ├── fopen
    ├── sprintf
    ├── rand
    ├── srand
    ├── strcpy
    ├── memset
    ├── strlen
    ├── wcscat
    ├── wcslen
    ├── __CxxFrameHandler
    ├── ??3@YAXPAX@Z
    ├── memcmp
    ├── _except_handler3
    ├── _local_unwind2
    ├── wcsrchr
    ├── swprintf
    ├── ??2@YAPAXI@Z
    ├── memcpy
    ├── strcmp
    ├── strrchr
    ├── __p___argv
    ├── __p___argc
    ├── _stricmp
    ├── free
    ├── malloc
    ├── ??0exception@@QAE@ABV0@@Z
    ├── ??1exception@@UAE@XZ
    ├── ??0exception@@QAE@ABQBD@Z
    ├── _CxxThrowException
    ├── calloc
    ├── strcat
    ├── _mbsstr
    ├── ??1type_info@@UAE@XZ
    ├── _exit
    ├── _XcptFilter
    ├── exit
    ├── _acmdln
    ├── __getmainargs
    ├── _initterm
    ├── __setusermatherr
    ├── _adjust_fdiv
    ├── __p__commode
    ├── __p__fmode
    ├── __set_app_type
    └── _controlfp

Mientras que el archivo sample_vg655_25th.exe muestra una cantidad normal de llamadas a funciones de sistema, sample_qwrty_dk2 presenta una alarmantemente baja cantidad de llamadas al sistema. Esto puede significar que sample_qwrty_dk2 es un ejecutable prácticamente vacío o está empaquetado de alguna manera.

**2. Obtenga la información de las secciones del PE Header. ¿Qué significa que algunas secciones tengan como parte de su nombre "upx"? Realice el procedimiento de desempaquetado para obtener las llamadas completas de las APIs.**

In [6]:
def inspect_pe_headers(name: str, executable):
    tree = Tree("[bold red]" + name)
    for section in executable.sections:
        tree.add("[purple]" + section.Name.decode('ascii'))
    
    print(tree)

In [7]:
inspect_pe_headers("sample_vg655_25th.exe", sample_vg655_25th)

sample_vg655_25th.exe
├── .text   
├── .rdata  
├── .data   
└── .rsrc   

In [8]:
inspect_pe_headers("sample_qwrty_dk2", sample_qwrty_dk2)

sample_qwrty_dk2
├── UPX0    
├── UPX1    
└── .rsrc   

Al observar los headers de sample_qwrty_dk2, se puede observar headers con nombres UPX. UPX hace referencia a *Ultimate Packer for eXecutables*. Como su nombre menciona, UPX hace referencia a un empaquetador de de software.

In [9]:
!cd MALWR && cp ./sample_qwrty_dk2 ./sample_qwrty_dk2_unpacked && upx -d sample_qwrty_dk2_unpacked

                       Ultimate Packer for eXecutables
                          Copyright (C) 1996 - 2023
UPX 4.0.2       Markus Oberhumer, Laszlo Molnar & John Reiser   Jan 30th 2023

        File size         Ratio      Format      Name
   --------------------   ------   -----------   -----------
      8192 <-      5632   68.75%    win32/pe     sample_qwrty_dk2_unpacked

Unpacked 1 file.


In [10]:
sample_qwrty_dk2_unpacked = pe.PE("./MALWR/sample_qwrty_dk2_unpacked")

In [11]:
inspect_pe_headers("sample_qwrty_dk2_unpacked", sample_qwrty_dk2_unpacked)

sample_qwrty_dk2_unpacked
├── .text   
├── .rdata  
├── .data   
└── .rsrc   

In [12]:
inspect_pe("sample_qwrty_dk2_unpacked", sample_qwrty_dk2_unpacked)

sample_qwrty_dk2_unpacked
├── KERNEL32.DLL
│   ├── CloseHandle
│   ├── WaitForSingleObject
│   ├── CreateEventA
│   ├── ExitThread
│   ├── Sleep
│   ├── GetComputerNameA
│   ├── CreatePipe
│   ├── DisconnectNamedPipe
│   ├── TerminateProcess
│   ├── WaitForMultipleObjects
│   ├── TerminateThread
│   ├── CreateThread
│   ├── CreateProcessA
│   ├── DuplicateHandle
│   ├── GetCurrentProcess
│   ├── ReadFile
│   ├── PeekNamedPipe
│   ├── SetEvent
│   ├── WriteFile
│   ├── SetProcessPriorityBoost
│   ├── SetThreadPriority
│   ├── GetCurrentThread
│   ├── SetPriorityClass
│   ├── lstrcatA
│   ├── lstrcpyA
│   ├── GetEnvironmentVariableA
│   ├── GetShortPathNameA
│   ├── GetModuleFileNameA
│   ├── GetStartupInfoA
│   └── GetModuleHandleA
├── MSVCRT.dll
│   ├── _controlfp
│   ├── _beginthread
│   ├── _strnicmp
│   ├── sprintf
│   ├── atol
│   ├── strchr
│   ├── free
│   ├── malloc
│   ├── _exit
│   ├── _XcptFilter
│   ├── exit
│   ├── _acmdln
│   ├── __getmainargs
│   ├── _initterm
│   ├── __setusermatherr
│   ├── _adjust_fdiv
│   ├── __p__commode
│   ├── __p__fmode
│   ├── __set_app_type
│   ├── _except_handler3
│   └── _itoa
├── SHELL32.dll
│   ├── ShellExecuteExA
│   └── SHChangeNotify
├── USER32.dll
│   └── LoadStringA
└── WS2_32.dll
    ├── htons
    ├── connect
    ├── socket
    ├── WSAStartup
    ├── send
    ├── inet_addr
    ├── recv
    └── closesocket

Una vez desempaquetado, se puede notar como el ejecutable sample_qwrty_dk2 ahora muestra una cantidad mucho mayor de llamadas a APIs del sistema. 

**3. Según el paper "Towards Understanding Malware Behaviour by the Extraction of API Calls", ¿en qué categoría sospechosas pueden clasificarse estos ejemplos en base a algunas de las llamadas a las APIs que realizan? Muestre una tabla con las APIs sospechosas y la categoría de malware que el paper propone.**

In [13]:
behaviours = [
    ("Search Files to Infect", {
        "FindClose", "FindFirstFile", "FindFirstFileEx", "FindFirstFileName", "TransactedW", 
        "FindFirstFileNameW", "FindFirstFileTransacted", "FindFirstStream", "TransactedW", 
        "FindFirstStreamW", "FindNextFile", "FindNextFileNameW", "FindNextStreamW", "SearchPath",
    }),
    
    ("Copy/Delete Files", {
        "CloseHandle", "CopyFile", "CopyFileEx", "CopyFileTransacted", "CreateFile", 
        "CreateFileTransacted", "CreateHardLink", "CreateHardLink", "Transacted", "CreateSymbolicLink", 
        "CreateSymbolic", "LinkTransacted", "DeleteFile", "DeleteFileTransacted",
    }),
    
    ("Get File Information", {
        "GetBinaryType", "GetCompressed", "FileSize", "GetCompressedFile", "SizeTransacted", 
        "GetFileAttributes", "GetFileAttributesEx", "GetFileAttributes", "Transacted", "GetFileBandwidth", 
        "Reservation", "GetFileInformation", "ByHandle", "GetFileInformation", "ByHandleEx", "GetFileSize", 
        "GetFileSizeEx", "GetFileType", "GetFinalPathName", "ByHandle", "GetFullPathName", "GetFullPathName", 
        "Transacted", "GetLongPathName", "GetLongPathName", "Transacted", "GetShortPathName", 
        "GetTempFileName", "GetTempPath",
    }),
    
    ("Move Files", {
        "MoveFile", "MoveFileEx", "MoveFileTransacted", "MoveFileWithProgress",
    }),
    
    ("Read/Write Files", {
        "OpenFile", "OpenFileById", "ReOpenFile", "ReplaceFile", "WriteFile", "CreateFile", "CloseHandle"
    }),
    
    ("Change File Attributes", {
        "SetFileApisToANSI", "SetFileApisToOEM", "SetFileAttributes", "SetFileAttributesTransacted", 
        "SetFileBandwidthReservation", "SetFileInformationByHandle", "SetFileShortName", "SetFileValidData"
    })
]

In [14]:
def classify_behaviour(name: str, executable):
    found_behaviours = set()
    
    for entry in executable.DIRECTORY_ENTRY_IMPORT:
        for function in entry.imports:
            found_behaviours.add(function.name.decode('ascii'))
    
    table = Table(title=name)
    table.add_column("Behaviour")
    table.add_column("Malware Category")
    table.add_column("API Function Calls")
    
    for i, behaviour in enumerate(behaviours):
        name, behaviour = behaviour
        matching_apis = behaviour.intersection(found_behaviours)
        
        if len(matching_apis) != 0:
            table.add_row(str(i+1), name, str(matching_apis))
    
    print(table)

In [15]:
classify_behaviour("sample_vg655_25th.exe", sample_vg655_25th)
classify_behaviour("sample_qwrty_dk2", sample_qwrty_dk2)
classify_behaviour("sample_qwrty_dk2_unpacked", sample_qwrty_dk2_unpacked)

                         sample_vg655_25th.exe                         
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Behaviour ┃ Malware Category     ┃ API Function Calls               ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2         │ Copy/Delete Files    │ {'CloseHandle'}                  │
│ 3         │ Get File Information │ {'GetFileSizeEx', 'GetFileSize'} │
│ 5         │ Read/Write Files     │ {'CloseHandle', 'WriteFile'}     │
└───────────┴──────────────────────┴──────────────────────────────────┘

                  sample_qwrty_dk2                   
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Behaviour ┃ Malware Category ┃ API Function Calls ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
└───────────┴──────────────────┴────────────────────┘

                   sample_qwrty_dk2_unpacked                    
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Behaviour ┃ Malware Category  ┃ API Function Calls           ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 2         │ Copy/Delete Files │ {'CloseHandle'}              │
│ 5         │ Read/Write Files  │ {'CloseHandle', 'WriteFile'} │
└───────────┴───────────────────┴──────────────────────────────┘

**4. Para el archivo "sample_vg655_25th.exe" obtenga el HASH en base al algoritmo SHA256.**

In [16]:
with open("./MALWR/sample_vg655_25th.exe", "rb") as file:
    print(file_digest(file, "sha256").hexdigest())

ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa

**5. Para el archivo "sample_vg655_25th.exe", ¿cuál es el propósito de la DLL ADVAPI32.dll?**

La DLL ADVAPI32.dll, también conocida como Advanced Windows 32 Base API permite al usuario:
- Modificar el Registro de Windows
- Apagar y enceder el dispositivo
- Iniciar, parar y crear Servicios
- Manejar a los usuarios

[StackOverflow (2021)](https://stackoverflow.com/questions/9194638/anyone-knows-what-does-advapi-stand-for)

**6. Para el archivo "sample_vg655_25th.exe", ¿cuál es el propósito de la API CryptReleaseContext?**

CryptReleaseContext es un *Cryptographic Service Provider*. Básicamente es un módulo de software encargado de utilizar criptografía para autenticar, encriptar y desencriptar.

[Microsoft, 2021](https://learn.microsoft.com/en-us/windows/win32/api/wincrypt/nf-wincrypt-cryptreleasecontext)

**7. Con la información recopilada hasta el momento, indique para el archivo "sample_vg655_25th.exe" si es sospechoso o no, y cuál podría ser su propósito.**

Tomando en cuenta sample_vg655_25th es capaz de encriptar, leer, escribir y modificar archivos; podemos concluir que el ejecutable es sopechoso de ser un malware. Específicamente, tenemos buenas razones para sospechar que sample_vg655_25th es un Ransomware. De manera que leería los archivos en tu computador y los encriptaría.

## Parte 2 - Análisis dinámico

**8. Utilice la plataforma de análisis dinámico https://www.hybrid-analysis.com y cargue el archivo "sample_vg655_25th.exe". ¿Se corresponde el HASH de la plataforma con el generado? ¿Cuál es el nombre del malware encontrado? ¿Cuál es el propósito de este malware?**

El [análisis generado](https://www.hybrid-analysis.com/sample/ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa) muestra el mismo Hash generado con el de la plataforma. Este malware tiene el nombre de owo_im_not_ransomware_xd.exe. 

Asimismo, la plataforma muestra que el malware efectivamente es un Ransomware.

**9. Muestre las capturas de pantalla sobre los mensajes que este malware presenta a usuario. ¿Se corresponden las sospechas con el análisis realizado en el punto 7?**

![desktop](https://www.hybrid-analysis.com/file-inline/60d9e5a39ffc8516db45537f/screenshot/screen_1.png)

![threat](https://www.hybrid-analysis.com/file-inline/60d9e5a39ffc8516db45537f/screenshot/screen_2.png)

Las siguientes capturas confirman las sospechas realizadas en el punto 7. Efectivamente, el Malware era un Ransomware.